#### Dimension table - Data Cleaning


##### Purpose of the noode book:
1. Clean duplicate, null values from dimension tables
2. Write to gold layer data storage

In [11]:
# Set the folder paths so that it can be used later
silver_folder_path = 'abfss://ws-container@bogomarketingdl.dfs.core.windows.net/silver'
gold_folder_path = 'abfss://ws-container@bogomarketingdl.dfs.core.windows.net/gold'

In [ ]:
%%sql
CREATE DATABASE IF NOT EXISTS gold_reward_program_spark
LOCATION 'abfss://ws-container@bogomarketingdl.dfs.core.windows.net/gold';

In [12]:
customer_profile = spark.read.parquet(f"{silver_folder_path}/customer_profile")

In [13]:
customer_profile.show()

In [14]:
from pyspark.sql.functions import col,isnan, when, count

In [17]:
customer_profile.filter(customer_profile.gender.isNull()).show()

In [16]:
customer_profile = customer_profile.na.fill("Unknown", subset=['gender'])
customer_profile = customer_profile.na.fill(0, subset=['income'])

In [18]:
customer_profile.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in ['gender', 'age', 'income']]
   ).show()

In [19]:
%%sql
Drop table if exists gold_reward_program_spark.customer_profile

In [20]:
customer_profile.write.mode("overwrite").format("parquet").saveAsTable("gold_reward_program_spark.customer_profile")

In [ ]:
offer_type = spark.read.parquet(f"{silver_folder_path}/offer_type")

In [ ]:
offer_type.show()

In [ ]:
from pyspark.sql import functions as F

# List of unique channels across all columns
channels = ['web', 'email', 'mobile', 'social']

# Iterate over each channel and create a dummy column for it
for channel in channels:
    # Check if the channel appears in any of the four channel columns
    offer_type = offer_type.withColumn(f'channel_{channel}', 
                       F.when(
                           (F.col('channel_1') == channel) |
                           (F.col('channel_2') == channel) |
                           (F.col('channel_3') == channel) |
                           (F.col('channel_4') == channel), 1
                       ).otherwise(0))

# Drop original channel columns if no longer needed
offer_type = offer_type.drop('channel_1', 'channel_2', 'channel_3', 'channel_4')

# Show the resulting DataFrame
offer_type.show()

In [ ]:
offer_type.write.mode("overwrite").format("parquet").saveAsTable("gold_reward_program_spark.offer_type")